# Tests with implemting numba to some of the processing functions

> Ricardo Peres, 26.07.2022

Numba: A High Performance Python Compiler: https://numba.pydata.org
 



In [4]:
import numba
import numpy as np

import sys
sys.path.append('../')
import pylars
from pylars.processing.waveforms import waveform_processing
from pylars.processing.peaks import peak_processing

#### Fire up a processor and get waveforms

In [5]:
process = pylars.processing.simple_processor(sigma_level=2, baseline_samples=50)
data_path = '/disk/gfs_atp/xenoscope/SiPMs/char_campaign/raw_data/run6/room_temp_21072022/LED_ON_300K_51_0V/Module0/LED_ON_300K_51_0V_Module_0_0.root'
process.load_raw_data(data_path, 49,300)

In [7]:
channel_data = process.raw_data.get_channel_data('wf0')
baseline = waveform_processing.get_baseline_rough(channel_data[0], 50)
std = waveform_processing.get_std_rough(channel_data[0], 50)
pks = peak_processing.find_peaks_simple(np.array(channel_data[0]), baseline, std, 2)

## Test area functions

In [8]:
def get_area(waveform: np.ndarray, baseline_value: float,
             peak_start: int, peak_end: int, dt: int = 10) -> float:
    """Get area of a single identified peak in a waveform.

    Args:
        waveform (_type_): _description_
        baseline_value (float): _description_
        peak_start (int): _description_
        peak_end (int): _description_
        dt (int, optional): _description_. Defaults to 10.

    Returns:
        _type_: _description_
    """
    peak_wf = waveform[peak_start:peak_end]
    area_under = np.sum(baseline_value - peak_wf) * 10
    return area_under

@numba.njit
def get_area_numba(waveform: np.ndarray, baseline_value: float,
             peak_start: int, peak_end: int, dt: int = 10) -> float:
    """Get area of a single identified peak in a waveform.

    Args:
        waveform (_type_): _description_
        baseline_value (float): _description_
        peak_start (int): _description_
        peak_end (int): _description_
        dt (int, optional): _description_. Defaults to 10.

    Returns:
        _type_: _description_
    """
    peak_wf = waveform[peak_start:peak_end]
    area_under = np.sum(baseline_value - peak_wf) * 10
    return area_under

In [9]:
_peak = pks[0]
_waveform = np.array(channel_data[0])

In [10]:
%%timeit
get_area(_waveform, baseline, _peak[0], _peak[-1])

9.11 µs ± 963 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
%%timeit
get_area_numba(_waveform, baseline, _peak[0], _peak[-1])

The slowest run took 17.30 times longer than the fastest. This could mean that an intermediate result is being cached.
3.66 µs ± 5.86 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
def get_all_areas_normal(waveform: np.ndarray, peaks: np.ndarray, baseline_value: float) -> np.ndarray:
    """Compute the areas of all the peaks in a waveform.
    TO DO: use np.apply_along_axis or similar and see if
    there is speed improvement.
    """
    areas = np.zeros(len(peaks))
    for i, _peak in enumerate(peaks):
        areas[i] = get_area(waveform, baseline_value, _peak[0], _peak[-1])
    return areas

def get_all_areas_list(waveform: np.ndarray, peaks: np.ndarray, baseline_value: float) -> list:
    """Compute the areas of all the peaks in a waveform.
    TO DO: use np.apply_along_axis or similar and see if
    there is speed improvement.
    """

    areas = [get_area(waveform, baseline_value, _peak[0], _peak[-1]) for _peak in peaks]

    return areas

def get_all_areas_numba(waveform: np.ndarray, peaks: np.ndarray, baseline_value: float) -> np.ndarray:
    """Compute the areas of all the peaks in a waveform.
    TO DO: use np.apply_along_axis or similar and see if
    there is speed improvement.
    """
    areas = np.zeros(len(peaks))
    for i, _peak in enumerate(peaks):
        areas[i] = get_area_numba(waveform, baseline_value, _peak[0], _peak[-1])
    return areas


In [13]:
%%timeit
areas = get_all_areas_normal(_waveform, pks, baseline)

31.4 µs ± 6.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
%%timeit
areas = get_all_areas_list(_waveform, pks, baseline)

29.5 µs ± 5.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
%%timeit
areas = get_all_areas_numba(_waveform, pks, baseline)

4.61 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Lengths

In [16]:
def get_all_lengths(peaks: list) -> list:
    """COmpute the lengths of all the peaks in a waveform.
    TO DO: try numba to speed it up.

    Args:
        peaks (_type_): _description_

    Returns:
        _type_: _description_
    """
    lengths = [len(_peak) for _peak in peaks]
    return lengths

@numba.njit
def get_all_lengths_numba(peaks: list) -> list:
    """COmpute the lengths of all the peaks in a waveform.
    TO DO: try numba to speed it up.

    Args:
        peaks (_type_): _description_

    Returns:
        _type_: _description_
    """
    lengths = [len(_peak) for _peak in peaks]
    return lengths

In [17]:
%%timeit
get_all_lengths(pks)

328 ns ± 26.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [18]:
%%timeit
get_all_lengths_numba(pks)

/disk/groups/atp/miniconda3/envs/sipms/lib/python3.8/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'peaks' of function 'get_all_lengths_numba'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../tmp/ipykernel_66840/3244411019.py", line 14:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


115 µs ± 38.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Positions

In [19]:
def get_all_positions(peaks: list) -> list:
    """Calcultes the initial position of the identified peak
    in number of samples.

    Args:
        peaks (_type_): array of identified peaks.

    Returns:
        _type_: list of positions of peaks.
    """
    positions = [_peak[0] for _peak in peaks]
    return positions

from numba.typed import List

@numba.njit
def get_all_positions_numba(peaks: list) -> list:
    """Calcultes the initial position of the identified peak
    in number of samples.

    Args:
        peaks (_type_): array of identified peaks.

    Returns:
        _type_: list of positions of peaks.
    """
    positions = List()
    [positions.append(_peak[0]) for _peak in peaks]
    return positions

In [20]:
%%timeit
pos = get_all_positions(pks)

711 ns ± 183 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%%timeit
pos_numba = get_all_positions_numba(pks)

/disk/groups/atp/miniconda3/envs/sipms/lib/python3.8/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'peaks' of function 'get_all_positions_numba'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../tmp/ipykernel_66840/1251832980.py", line 16:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


81.3 µs ± 51.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## split peaks

In [22]:
def split_consecutive_normal(array: np.array, stepsize: int = 1):
    split_index = np.where(np.diff(array) != stepsize)[0] + 1
    split_array = np.split(array, split_index)
    return split_array

@numba.njit
def split_consecutive_numba(array: np.array, stepsize: int = 1):
    split_index = np.where(np.diff(array) != stepsize)[0] + 1
    split_array = np.split(array, split_index)
    return split_array

def split_consecutive_kindapython(array: np.array, stepsize: int = 1):
    split_list = list()
    _running_list = list()
    for i, index_number_from_waveform in enumerate(array):
        if i==(len(array)-1):
            if index_number_from_waveform ==array[i-1]+1:
                _running_list.append(index_number_from_waveform)
                split_list.append(_running_list)
            else:
                split_list.append([index_number_from_waveform])
            break
        if index_number_from_waveform == (array[i+1]-1):
            _running_list.append(index_number_from_waveform)
        else:
            _running_list.append(index_number_from_waveform)
            split_list.append(_running_list)
            _running_list = list()
#    split_index = np.where(np.diff(array) != stepsize)[0] + 1
#    split_array = np.split(array, split_index)
    return split_list

In [23]:
bellow_baseline = np.where(_waveform < (baseline - std*1.5))[0]

In [29]:
%%timeit
split_array = split_consecutive_normal(bellow_baseline)

34.8 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
%%timeit
split_array_numba = split_consecutive_numba(bellow_baseline)

71.8 µs ± 8.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
%%timeit
split_list = split_consecutive_kindapython(bellow_baseline)

34.2 µs ± 4.98 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
